#Reliability Re-Annotation Process

In [ ]:
#load GOLD Standard Corpus
trainDF['text'] = trainDF['text'].replace('\n','', regex=True)
trainDF['text'] = trainDF['text'].replace('\n',' ', regex=True)
trainDF['text'] = trainDF['text'].replace(r'\\n',' ', regex=True)

trainDF

,text,label
0,"Sustained effort, adequate thinking on the ice will go a long way.Its not open heart surgery for Gods sake.",FAKE
1,Are those all choices 100%? Heart disease is a choice? Why not go furth,FAKE
2,How about if she died of a heart attack from Pfizer?Pretty much the same risk...,FAKE
3,Brushing your teeth 3x per day [statistically] significantly reduces your risk of heart attack. So there.,FAKE
4,"but hey, bo burnham. and his hands i had just had my heart surgery like a month before but i was SPR",FAKE
...,...,...
470,Some tests can diagnose coronary heart disease,TRUE
471,"A treadmill test (or exercise stress test) can help diagnose atherosclerosis, or the narrowing of the heart's arteries.",TRUE
472,"A treadmill test (or exercise stress test) can not help diagnose atherosclerosis, or the narrowing of the heart's arteries.",FAKE
473,"Other tests include a radionucleotide myocardial perfusion stress test, This can also help diagnose a narrowing of the arteries in your heart.",TRUE


##Train Gold Corpus

In [ ]:
from sklearn import model_selection
train_x,test_x,train_y,test_y = model_selection.train_test_split(trainDF,trainDF,test_size=0.15,random_state=11)
train_x.reset_index(drop=True,inplace=True)
test_x.reset_index(drop=True,inplace=True)
train_x.shape

(403, 2)

###Build Flair GOLD *Corpus*

In [ ]:
from flair.data import Corpus
from flair.datasets import SentenceDataset
from flair.data import Sentence

train_labeled=[]
for i in range(len(train_x['text'])):
    sentence = Sentence(train_x['text'][i]).add_label('reliability', train_x['label'][i])
    train_labeled.append(sentence)

test_labeled=[]
for i in range(len(test_x['text'])):
    sentence = Sentence(test_x['text'][i]).add_label('reliability', test_x['label'][i])
    test_labeled.append(sentence)


# training dataset consisting of four sentences (2 labeled as "food" and 2 labeled as "drink")
train = SentenceDataset(train_labeled)
test = SentenceDataset(test_labeled)

# make a corpus with train and test split
corpus = Corpus(train=train, test=test)

print(len(corpus.test))

72


In [ ]:
print(len(corpus.test))
print(len(corpus.train))

print(len(test_labeled))
print(len(train_labeled))

72
363
72
403


###Training 

In [ ]:
from flair.data import Corpus
#from flair.datasets import TREC_6
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [ ]:
#this code is used in this notebook of gold training

In [ ]:
from flair.trainers import ModelTrainer
from flair.models.text_classification_model import TARSClassifier


# 1. load base TARS
tars = TARSClassifier.load('tars-base')

# 2. make the model aware of the desired set of labels from the new corpus
tars.add_and_switch_to_new_task("TRUE_FAKE", label_dictionary=corpus.make_label_dictionary())

# 3. initialize the text classifier trainer with your corpus
trainer = ModelTrainer(tars, corpus)

# 4. train model
trainer.train(base_path='result/gold', # path to store the model artifacts
              learning_rate=0.02, # use very small learning rate
              mini_batch_size=1, # small mini-batch size since corpus is tiny
              max_epochs=10, # terminate after 10 epochs
              train_with_dev=True,
              )

2021-05-11 20:24:45,485 https://nlp.informatik.hu-berlin.de/resources/models/tars-base/tars-base-v8.pt not found in cache, downloading to /tmp/tmpub16jujd


100%|██████████| 438064585/438064585 [00:11<00:00, 37026305.95B/s]

2021-05-11 20:24:57,393 copying /tmp/tmpub16jujd to cache at /root/.flair/models/tars-base-v8.pt


2021-05-11 20:24:59,097 removing temp file /tmp/tmpub16jujd
2021-05-11 20:24:59,322 loading file /root/.flair/models/tars-base-v8.pt



init TARS
2021-05-11 20:25:08,525 Computing label dictionary. Progress:


100%|██████████| 284/284 [00:00<00:00, 17606.46it/s]

2021-05-11 20:25:08,557 [b'TRUE', b'FAKE']
2021-05-11 20:25:08,562 ----------------------------------------------------------------------------------------------------
2021-05-11 20:25:08,569 Model: "TARSClassifier(
  (document_embeddings): None
  (decoder): None
  (loss_function): None
  (tars_model): TextClassifier(
    (document_embeddings): TransformerDocumentEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bia

2021-05-11 20:26:10,507 epoch 1 - iter 26/263 - loss 1.25209515 - samples/sec: 0.42 - lr: 0.020000
2021-05-11 20:26:59,774 epoch 1 - iter 52/263 - loss 1.04188734 - samples/sec: 0.53 - lr: 0.020000
2021-05-11 20:27:51,533 epoch 1 - iter 78/263 - loss 0.87507077 - samples/sec: 0.50 - lr: 0.020000
2021-05-11 20:28:39,133 epoch 1 - iter 104/263 - loss 1.00379624 - samples/sec: 0.55 - lr: 0.020000
2021-05-11 20:29:27,389 epoch 1 - iter 130/263 - loss 0.98903180 - samples/sec: 0.54 - lr: 0.020000
2021-05-11 20:30:12,427 epoch 1 - iter 156/263 - loss 0.99714494 - samples/sec: 0.58 - lr: 0.020000
2021-05-11 20:31:09,433 epoch 1 - iter 182/263 - loss 0.94946873 - samples/sec: 0.46 - lr: 0.020000
2021-05-11 20:31:59,022 epoch 1 - iter 208/263 - loss 0.87785259 - samples/sec: 0.52 - lr: 0.020000
2021-05-11 20:32:53,282 epoch 1 - iter 234/263 - loss 0.81895495 - samples/sec: 0.48 - lr: 0.020000
2021-05-11 20:33:58,793 epoch 1 - iter 260/263 - loss 0.80726215 - samples/sec: 0.40 - lr: 0.020000
202

{'dev_loss_history': [],
 'dev_score_history': [],
 'test_score': 0.8723,
 'train_loss_history': [0.8221295834562127,
  0.473281289617286,
  0.2530965366117597,
  0.12739053387276347,
  0.06423078877861879,
  0.057067517836428744,
  0.04281600573784145,
  0.0590338192364401,
  0.04207156732331118,
  0.04674250481638282]}

# annotate sample curpus

In [ ]:
trainDF

,text,label
0,"Sustained effort, adequate thinking on the ice will go a long way.\nIts not open heart surgery for Gods sake.",FAKE
1,Are those all choices 100%? Heart disease is a choice? Why not go furth,FAKE
2,How about if she died of a heart attack from Pfizer?\nPretty much the same risk...\n\n,FAKE
3,Brushing your teeth 3x per day [statistically] significantly reduces your risk of heart attack. \nSo there.,FAKE
4,"but hey, bo burnham. and his hands i had just had my heart surgery like a month before but i was SPR",FAKE
...,...,...
4805,"take your prescribed medicines every day, even when you’re feeling great",TRUE
4806,you should keep your blood pressure under control – that is less than 130/80 mmHg,TRUE
4807,effects of alcohol consumption on heart health are variable,TRUE
4808,littlee alcohol drinking reduces your risk of having a heart attack by about 25 percent,TRUE


In [ ]:
sample_sent=[]
for i in range(len(trainDF['text'])):
    sentence = Sentence(trainDF['text'][i])
    sample_sent.append(sentence)
    
sample_sent

In [ ]:
sample_annotated=[]
for i in range(len(trainDF['text'])):
    classifier.predict(sample_sent[i])
    sample_annotated.append(sample_sent[i].labels)

sample_annotated


In [ ]:
trainDF['sample_annotated']=sample_annotated


,text,label,sample_annotated
0,"Sustained effort, adequate thinking on the ice will go a long way.\nIts not open heart surgery for Gods sake.",FAKE,[FAKE (0.9999)]
1,Are those all choices 100%? Heart disease is a choice? Why not go furth,FAKE,[FAKE (0.3262)]
2,How about if she died of a heart attack from Pfizer?\nPretty much the same risk...\n\n,FAKE,[FAKE (1.0)]
3,Brushing your teeth 3x per day [statistically] significantly reduces your risk of heart attack. \nSo there.,FAKE,[FAKE (0.9945)]
4,"but hey, bo burnham. and his hands i had just had my heart surgery like a month before but i was SPR",FAKE,[FAKE (0.9999)]
...,...,...,...
4805,"take your prescribed medicines every day, even when you’re feeling great",TRUE,[TRUE (0.9998)]
4806,you should keep your blood pressure under control – that is less than 130/80 mmHg,TRUE,[TRUE (1.0)]
4807,effects of alcohol consumption on heart health are variable,TRUE,[TRUE (0.9998)]
4808,littlee alcohol drinking reduces your risk of having a heart attack by about 25 percent,TRUE,[TRUE (0.9999)]


In [ ]:
    
sample_sent[0].labels[0].to_dict()['value']
sample_sent[0].labels[0].to_dict()['confidence']


0.9999387264251709

In [ ]:
scory=[]
labelnew=[]
for i in range(len(trainDF['text'])):
    scory.append(sample_sent[i].labels[0].to_dict()['confidence'])
    labelnew.append(sample_sent[i].labels[0].to_dict()['value'])

trainDF['labelnew']=labelnew
trainDF['scory']=scory
trainDF

,text,label,sample_annotated,labelnew,scory
0,"Sustained effort, adequate thinking on the ice will go a long way.\nIts not open heart surgery for Gods sake.",FAKE,[FAKE (0.9999)],FAKE,0.999939
1,Are those all choices 100%? Heart disease is a choice? Why not go furth,FAKE,[FAKE (0.3262)],FAKE,0.326175
2,How about if she died of a heart attack from Pfizer?\nPretty much the same risk...\n\n,FAKE,[FAKE (1.0)],FAKE,0.999960
3,Brushing your teeth 3x per day [statistically] significantly reduces your risk of heart attack. \nSo there.,FAKE,[FAKE (0.9945)],FAKE,0.994510
4,"but hey, bo burnham. and his hands i had just had my heart surgery like a month before but i was SPR",FAKE,[FAKE (0.9999)],FAKE,0.999936
...,...,...,...,...,...
4805,"take your prescribed medicines every day, even when you’re feeling great",TRUE,[TRUE (0.9998)],TRUE,0.999840
4806,you should keep your blood pressure under control – that is less than 130/80 mmHg,TRUE,[TRUE (1.0)],TRUE,0.999965
4807,effects of alcohol consumption on heart health are variable,TRUE,[TRUE (0.9998)],TRUE,0.999820
4808,littlee alcohol drinking reduces your risk of having a heart attack by about 25 percent,TRUE,[TRUE (0.9999)],TRUE,0.999938


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

cm = confusion_matrix(trainDF['label'], trainDF['labelnew'])
print(cm)

cr = classification_report(trainDF['label'], trainDF['labelnew'])
print(cr)

[[1709  606]
 [  53 2442]]
              precision    recall  f1-score   support

        FAKE       0.97      0.74      0.84      2315
        TRUE       0.80      0.98      0.88      2495

    accuracy                           0.86      4810
   macro avg       0.89      0.86      0.86      4810
weighted avg       0.88      0.86      0.86      4810

